# Loop 1 LB Feedback Analysis

LB Score: 70.6761 (matches CV exactly)
Target: 68.922808
Gap: 1.753 points

This notebook analyzes all available solutions to find the best ensemble.

In [ ]:
import numpy as np
import pandas as pd
import math
from numba import njit
import glob
import os

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a group of trees"""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    """Strip 's' prefix from values"""
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

tx, ty = make_polygon_template()
print(f"Tree template: {len(tx)} vertices")

In [ ]:
# List all available solution files
solution_dirs = [
    '/home/code/santa-2025-csv/',
    '/home/code/bucket-of-chump/',
    '/home/code/telegram-solution/',
    '/home/code/telegram-full/',
    '/home/code/best-public/',
    '/home/code/best-public-packed/',
]

all_files = []
for d in solution_dirs:
    if os.path.exists(d):
        all_files.extend(glob.glob(d + '*.csv'))

print(f"Found {len(all_files)} solution files:")
for f in all_files:
    print(f"  {f}")

In [ ]:
# Score each solution file and find best for each N
best = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}
file_scores = {}

for fp in all_files:
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        print(f"Error reading {fp}: {e}")
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        print(f"Skipping {fp}: missing columns")
        continue
    
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    total_score = 0
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        sc = score_group(xs, ys, ds, tx, ty)
        total_score += sc
        
        if sc < best[n]['score']:
            best[n]['score'] = float(sc)
            best[n]['data'] = g.drop(columns=['N']).copy()
            best[n]['src'] = os.path.basename(fp)
    
    file_scores[os.path.basename(fp)] = total_score
    print(f"{os.path.basename(fp)}: total score = {total_score:.6f}")

print("\nFile scores sorted:")
for fn, sc in sorted(file_scores.items(), key=lambda x: x[1]):
    print(f"  {fn}: {sc:.6f}")

In [ ]:
# Calculate ensemble score
ensemble_score = sum(best[n]['score'] for n in range(1, 201))
print(f"\nEnsemble score (best from each N): {ensemble_score:.6f}")
print(f"Target score: 68.922808")
print(f"Gap to target: {ensemble_score - 68.922808:.6f}")

# Show which sources contribute to the ensemble
source_counts = {}
for n in range(1, 201):
    src = best[n]['src']
    source_counts[src] = source_counts.get(src, 0) + 1

print(f"\nSource contributions:")
for src, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f"  {src}: {count} N values")

In [ ]:
# Analyze which N values have the most room for improvement
print("\nTop 30 N values with highest contribution to score:")
n_scores = [(n, best[n]['score'], best[n]['src']) for n in range(1, 201)]
n_scores.sort(key=lambda x: -x[1])

for n, score, src in n_scores[:30]:
    print(f"  N={n:3d}: score={score:.6f} (from {src})")

In [ ]:
# Create ensemble submission
rows = []
for n in range(1, 201):
    if best[n]['data'] is not None:
        rows.append(best[n]['data'])

ensemble_df = pd.concat(rows, ignore_index=True)
print(f"Ensemble submission rows: {len(ensemble_df)}")
print(f"Expected rows: {sum(range(1, 201))}")

# Save ensemble
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print("\nEnsemble saved to /home/submission/submission.csv")

In [ ]:
# Verify the ensemble score
df = pd.read_csv('/home/submission/submission.csv')
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

total = 0
for n, g in df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    sc = score_group(xs, ys, ds, tx, ty)
    total += sc

print(f"Verified ensemble score: {total:.6f}")

In [ ]:
# Summary
print("="*60)
print("SUMMARY")
print("="*60)
print(f"Best single file: {min(file_scores.items(), key=lambda x: x[1])}")
print(f"Ensemble score: {ensemble_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {ensemble_score - 68.922808:.6f}")
print("="*60)